### Using supabase as vector store

In [21]:
import logging
import sys
import os
# Uncomment to see debug logs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import SimpleDirectoryReader, Document, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.supabase import SupabaseVectorStore
import textwrap

In [22]:
documents = SimpleDirectoryReader("data\paul_graham\\").load_data()

In [23]:
from dotenv import load_dotenv
load_dotenv('..\..\.env')

True

In [24]:
# creating embeddings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [29]:
vector_store = SupabaseVectorStore(
    postgres_connection_string=(
       f"postgresql://{os.getenv('SUPABASE_USER')}:{os.getenv('SUPABASE_PASSWORD')}@{os.getenv('SUPABASE_HOST')}:5432/postgres"
    ),
    collection_name="base_demo",
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

# in case of error look into table schema in supabase

# create table vecs.base_demo (
#   id varchar primary key,
#   vec vector(384),
#   metadata jsonb
# );


In [30]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.embeddings import resolve_embed_model
from llama_index.llms.ollama import Ollama

# ollama
Settings.llm = Ollama(model="llama2", request_timeout=60.0)

In [31]:
query_engine = index.as_query_engine()
response = query_engine.query("Who is the author?")
print(textwrap.fill(str(response), 100))

The author of the given text is Paul Graham.


In [32]:
response = query_engine.query("What did the author do growing up?")
print(textwrap.fill(str(response), 100))

Based on the context provided in the essay, the author worked on writing and programming outside of
school during their youth. They specifically mention that they wrote short stories and tried to
write essays, but couldn't remember any programs they wrote due to the limited technology available
at the time. The author also mentions that they didn't plan to study programming in college, but
instead wanted to study philosophy. However, they eventually switched to AI after being influenced
by a novel and a PBS documentary.
